In [1]:
import requests, httplib, re, json, getpass

In [2]:
from IPython.display import HTML

In [ ]:
core_services_url = 'https://192.168.126.158:8443/www/core-service/services/listServices'
manager_services_url = 'https://192.168.126.158:8443/www/manager-service/services/listServices'
login_request = 'https://esm:8443/www/core-service/rest/LoginService/login?login=admin&password={}'.format(getpass.getpass())

In [ ]:
login_request_json = 'https://esm:8443/www/core-service/rest/LoginService/login?login=admin&password={}&alt=json'.format(getpass.getpass())

In [119]:
def doLogin(manager_host_name, user_name):
    login_request_json = 'https://{}:8443/www/core-service/rest/LoginService/login?login={}&password={}&alt=json'\
    .format(manager_host_name, user_name, getpass.getpass())
    login_object_json = requests.get(login_request_json, verify=False)
    json_response = json.loads(login_object_json.content)
    token = json_response.values()[0]['log.return']
    return token
def doLogout(manager_host_name, user_name, token):
    logout_request = 'https://{}:8443/www/core-service/rest/LoginService/logout?login={}&authToken={}'\
    .format(manager_host_name, user_name, token)
    run_logout=requests.get(logout_request, verify=False)
    if run_logout.content == '':
        return 'Logged out user: {}'.format(user_name)
    else:
        return 'COuld not log out user: {}, consult manager log or client request log for further information.'
def getSession(manager_host_name, authToken):
    session_url = 'https://{}:8443/www/core-service/rest/LoginService/getSession?authToken={}'.format(manager_host_name, authToken)
    session_token_request = requests.get(session_url, authToken, verify=False)
    auth_token_stripper = re.compile('\<authToken\>(\S+)\</authToken\>')
    creationMillis_Stripper = re.compile('\<creationMillis\>(\d+)\</creationMillis\>')
    expirationMillis_Stripper = re.compile('\<expirationMillis\>(\d+)\</expirationMillis\>')
    userId_stripper = re.compile('\<userId\>(\d+)\</userId\>')
    authToken = auth_token_stripper.findall(session_token_request.content)[0]
    creationMillis = creationMillis_Stripper.findall(session_token_request.content)[0]
    expirationMillis = expirationMillis_Stripper.findall(session_token_request.content)[0]
    userId = userId_stripper.findall(session_token_request.content)[0]
    sessionToken = authToken + creationMillis + expirationMillis + userId
    return authToken, creationMillis, expirationMillis, userId, sessionToken
def queryCases(manager_host_name, authToken):
    '''Returns All Case IDs'''
    resource_request_json = 'https://{}:8443/www/manager-service/rest/CaseService/findAllIds?&authToken={}&alt=json'\
    .format(manager_host_name, authToken)
    print(resource_request_json)
    retrieve_list = requests.get(resource_request_json, verify=False)
    response = json.loads(retrieve_list.content)
    return response[u'cas.findAllIdsResponse']['cas.return']
def queryReports(manager_host_name, authToken):
    '''No error, no return'''
    resource_request_json = 'https://{}:8443/www/manager-service/rest/ReportService/findAllIds?&authToken={}&alt=json'\
    .format(manager_host_name, authToken)
    print(resource_request_json)
    retrieve_list = requests.get(resource_request_json, verify=False)
    response = json.loads(retrieve_list.content)
    return response[u'rep.findAllIdsResponse']['rep.return']
def queryActiveLists(manager_host_name, authToken):
    '''No error, no return'''
    resource_request_json = 'https://{}:8443/www/manager-service/rest/ActiveListService/findAllIds?&authToken={}&alt=json'\
    .format(manager_host_name, authToken)
    print(resource_request_json)
    retrieve_list = requests.get(resource_request_json, verify=False)
    response = json.loads(retrieve_list.content)
    return response['act.findAllIdsResponse']['act.return']
def queryActiveListByID(manager_host_name, authToken, resourceID):
    resource_request_json = 'https://{}:8443/www/manager-service/rest/ActiveListService/getResourceById?resourceId={}&authToken={}&alt=json'\
    .format(manager_host_name, resourceID, authToken)
    print(resource_request_json)
    retrieve_list = requests.get(resource_request_json, verify=False)
    response = json.loads(retrieve_list.content)
    return response['act.getResourceByIdResponse']['act.return']
def getListEntries(manager_host_name, authToken, resourceID):
    resource_request_json = \
    'https://{}:8443/www/manager-service/rest/ActiveListService/getEntries?resourceId={}&authToken={}&alt=json'\
    .format(manager_host_name, resourceID, authToken)
    print(resource_request_json)
    retrieve_list = requests.get(resource_request_json, verify=False)
    #return retrieve_list
    response = json.loads(retrieve_list.content)
    return response['act.getEntriesResponse']['act.return']
def GenericAPICall(manager_host_name, Service, Call, authToken, parameter_kv_pairs=None, json_return=True):
    if parameter_kv_pairs==None:
        resource_request_json = \
        'https://{}:8443/www/manager-service/rest/{}/{}?&authToken={}&alt=json'\
        .format(manager_host_name, Service, Call, authToken) 
    else:
        paramaters=''
        parameter_kv_pairs = parameter_kv_pairs.split(',')
        print(parameter_kv_pairs)
        kv_pairs = {}
        for pair in parameter_kv_pairs:
            k = pair.split('=', 1)
            kv_pairs[k[0]] = k[1]
        kv_pairs['authToken'] = authToken
        for entry in kv_pairs.keys():
            paramaters +='&' + entry + '=' + kv_pairs[entry]
        print(kv_pairs)
        resource_request_json = \
        'https://{}:8443/www/manager-service/rest/{}/{}?{}&alt=json'\
        .format(manager_host_name, Service, Call, paramaters)
    print(resource_request_json)
    retrieve_list = requests.get(resource_request_json, verify=False)
    if not json_return:
        return retrieve_list
    else:
        response = json.loads(retrieve_list.content)
        return response
def getRuleContent():
    pass

In [4]:
service_dict = {
    'NetworkService':{
        'getTargetsWithRelationshipTypeForResourceById':{},
        'getTargetsByRelationshipForSourceId':{},
        'getSourcesWithThisTargetByRelationshipCount':{},
        'deleteResource':{},
        'getSourceURIWithThisTargetByRelatiobnshipForResourceId':{},
        'getAllAttachmentOnlyResourceIDs':{},
        'getNamesAndAliases':{},
        'containsDirectMemberByNameOrAlias':{},
        'getTargetsWithRelationshipTypeForResource':{},
        'getReverseRelationshipsOfThisAndParents':{},
        'getPersonalResourceRoots':{},
        'getESMVersion':{},
        'getResourceByName':{},
        'hasXPermission':{},
        'findAllIds':{},
        'addRelationship':{},
        'getReverseRelationshipsOfParents':{},
        'getResourcesReferencePages':{},
        'containsDirectMemberByName':{},
        'getTargetsAsURIByRelationshipForSourceId':{},
        'hasWritePermission':{},
        'deleteResources':{},
        'resolveRelationship':{},
        'getAllPathsToRoot':{},
        'getResourcesWithVisibilityToUsers':{},
        'getReferencePages':{},
        'findAll':{},
        'getExclusivelyDependentResources':{},
        'getAllowedUserTypes':{},
        'getResourcesByNameSafely':{},
        'getResourcesByNames':{},
        'getSourceURIWithThisTargetByRelatiobnship':{},
        'getRelationshipsOfParents':{},
        'getMetaGroupID':{},
        'isValidResourceID':{},
        'getServiceMajorVersion':{},
        'hasReverseRelationship':{},
        'containsDirectMemberByName1':{},
        'getChildNamesAndAliases':{},
        'getResourcesByIds':{},
        'getResourceTypesVisibleToUsers':{},
        'update':{},
        'getRelationshipsOfThisAndParents':{},
        'getPersonalAndSharedResourceRoots':{},
        'getSourcesWithThisTargetByRelationship':{},
        'containsDirectMemberByNameOrAlias1':{},
        'hasReadPermission':{},
        'copyResourceIntoGroup':{},
        'deleteByLocalId':{},
        'getDependentResourceIDsForResourceId':{},
        'getResourceIfModified':{},
        'findById':{},
        'getSourcesWithThisTargetByRelationshipForResourceId':{},
        'getSourcesWithThisTargetByACLRelationship':{},
        'delete':{},
        'getAllPathsToRootAsStrings':{},
        'loadAdditional':{},
        'insertResource':{},
        'insertResources':{},
        'getAllUnassignedResourceIDs':{},
        'getEnabledResourceIDs':{},
        'getSourceURIWithThisTargetByRelationship':{},
        'getResourceById':{},
        'updateACLForResourceById':{},
        'getCorruptedResources':{},
        'unloadAdditional':{},
        'isDisabled':{},
        'getTargetsAsURIByRelationship':{},
        'updateResources':{},
        'deleteByUUID':{},
        'getTargetsByRelationship':{},
        'getSourceURIWithThisTargetByRelationshipForResourceId':{},
        'getTargetsByRelationshipCount':{},
        'getPersonalGroup':{},
        'findByUUID':{},
        'resetState':{},
        'insert':{},
        'getServiceMinorVersion':{},
    },
    'ViewerConfigurationService':{
        'getTargetsWithRelationshipTypeForResourceById':{},
        'getTargetsByRelationshipForSourceId':{},
        'getSourcesWithThisTargetByRelationshipCount':{},
        'deleteResource':{},
        'getSourceURIWithThisTargetByRelatiobnshipForResourceId':{},
        'getAllAttachmentOnlyResourceIDs':{},
        'getNamesAndAliases':{},
        'containsDirectMemberByNameOrAlias':{},
        'getTargetsWithRelationshipTypeForResource':{},
        'getReverseRelationshipsOfThisAndParents':{},
        'getPersonalResourceRoots':{},
        'getESMVersion':{},
        'getResourceByName':{},
        'hasXPermission':{},
        'findAllIds':{},
        'addRelationship':{},
        'getReverseRelationshipsOfParents':{},
        'getResourcesReferencePages':{},
        'containsDirectMemberByName':{},
        'getTargetsAsURIByRelationshipForSourceId':{},
        'hasWritePermission':{},
        'deleteResources':{},
        'resolveRelationship':{},
        'getAllPathsToRoot':{},
        'getResourcesWithVisibilityToUsers':{},
        'getReferencePages':{},
        'findAll':{},
        'getExclusivelyDependentResources':{},
        'getAllowedUserTypes':{},
        'getResourcesByNameSafely':{},
        'getResourcesByNames':{},
        'getSourceURIWithThisTargetByRelatiobnship':{},
        'getRelationshipsOfParents':{},
        'getMetaGroupID':{},
        'isValidResourceID':{},
        'getServiceMajorVersion':{},
        'hasReverseRelationship':{},
        'containsDirectMemberByName1':{},
        'getChildNamesAndAliases':{},
        'getResourcesByIds':{},
        'getResourceTypesVisibleToUsers':{},
        'update':{},
        'getRelationshipsOfThisAndParents':{},
        'getPersonalAndSharedResourceRoots':{},
        'getSourcesWithThisTargetByRelationship':{},
        'containsDirectMemberByNameOrAlias1':{},
        'hasReadPermission':{},
        'copyResourceIntoGroup':{},
        'deleteByLocalId':{},
        'getDependentResourceIDsForResourceId':{},
        'getResourceIfModified':{},
        'findById':{},
        'getSourcesWithThisTargetByRelationshipForResourceId':{},
        'getSourcesWithThisTargetByACLRelationship':{},
        'delete':{},
        'getAllPathsToRootAsStrings':{},
        'loadAdditional':{},
        'insertResource':{},
        'insertResources':{},
        'getEnabledResourceIDs':{},
        'getAllUnassignedResourceIDs':{},
        'getSourceURIWithThisTargetByRelationship':{},
        'getResourceById':{},
        'updateACLForResourceById':{},
        'getCorruptedResources':{},
        'unloadAdditional':{},
        'isDisabled':{},
        'getTargetsAsURIByRelationship':{},
        'updateResources':{},
        'deleteByUUID':{},
        'getTargetsByRelationship':{},
        'getSourceURIWithThisTargetByRelationshipForResourceId':{},
        'getTargetsByRelationshipCount':{},
        'getPersonalGroup':{},
        'getViewerConfigurationIfNewer':{},
        'findByUUID':{},
        'resetState':{},
        'insert':{},
        'getServiceMinorVersion':{},
    },
    'DashboardService':{
        'getTargetsWithRelationshipTypeForResourceById':{},
        'getTargetsByRelationshipForSourceId':{},
        'getDataMonitorDataIfNewer':{},
        'getSourcesWithThisTargetByRelationshipCount':{},
        'deleteResource':{},
        'getSourceURIWithThisTargetByRelatiobnshipForResourceId':{},
        'getAllAttachmentOnlyResourceIDs':{},
        'getNamesAndAliases':{},
        'containsDirectMemberByNameOrAlias':{},
        'getTargetsWithRelationshipTypeForResource':{},
        'getReverseRelationshipsOfThisAndParents':{},
        'getPersonalResourceRoots':{},
        'getESMVersion':{},
        'getResourceByName':{},
        'hasXPermission':{},
        'findAllIds':{},
        'addRelationship':{},
        'getReverseRelationshipsOfParents':{},
        'getResourcesReferencePages':{},
        'containsDirectMemberByName':{},
        'getTargetsAsURIByRelationshipForSourceId':{},
        'hasWritePermission':{},
        'deleteResources':{},
        'resolveRelationship':{},
        'getAllPathsToRoot':{},
        'getResourcesWithVisibilityToUsers':{},
        'getReferencePages':{},
        'findAll':{},
        'getExclusivelyDependentResources':{},
        'getAllowedUserTypes':{},
        'getDashboardIfNewer':{},
        'getResourcesByNameSafely':{},
        'getResourcesByNames':{},
        'getSourceURIWithThisTargetByRelatiobnship':{},
        'getRelationshipsOfParents':{},
        'getMetaGroupID':{},
        'isValidResourceID':{},
        'getServiceMajorVersion':{},
        'hasReverseRelationship':{},
        'containsDirectMemberByName1':{},
        'getChildNamesAndAliases':{},
        'getResourcesByIds':{},
        'getResourceTypesVisibleToUsers':{},
        'update':{},
        'getRelationshipsOfThisAndParents':{},
        'getPersonalAndSharedResourceRoots':{},
        'getSourcesWithThisTargetByRelationship':{},
        'containsDirectMemberByNameOrAlias1':{},
        'hasReadPermission':{},
        'copyResourceIntoGroup':{},
        'getDependentResourceIDsForResourceId':{},
        'deleteByLocalId':{},
        'getResourceIfModified':{},
        'findById':{},
        'getSourcesWithThisTargetByRelationshipForResourceId':{},
        'getSourcesWithThisTargetByACLRelationship':{},
        'delete':{},
        'getAllPathsToRootAsStrings':{},
        'loadAdditional':{},
        'insertResource':{},
        'insertResources':{},
        'getEnabledResourceIDs':{},
        'getAllUnassignedResourceIDs':{},
        'getSourceURIWithThisTargetByRelationship':{},
        'getResourceById':{},
        'updateACLForResourceById':{},
        'getCorruptedResources':{},
        'unloadAdditional':{},
        'isDisabled':{},
        'getTargetsAsURIByRelationship':{},
        'updateResources':{},
        'deleteByUUID':{},
        'getTargetsByRelationship':{},
        'getSourceURIWithThisTargetByRelationshipForResourceId':{},
        'getTargetsByRelationshipCount':{},
        'getPersonalGroup':{},
        'findByUUID':{},
        'resetState':{},
        'insert':{},
        'getDataMonitorDatasIfNewer':{},
        'getServiceMinorVersion':{},
    },
    'DataMonitorQoSService':{
        'disableQoSConstraintsOnDM':{},
        'enableQoSConstraintsOnDM':{},
    },
    'DrilldownService':{
        'getTargetsWithRelationshipTypeForResourceById':{},
        'getTargetsByRelationshipForSourceId':{},
        'getSourcesWithThisTargetByRelationshipCount':{},
        'deleteResource':{},
        'getSourceURIWithThisTargetByRelatiobnshipForResourceId':{},
        'getAllAttachmentOnlyResourceIDs':{},
        'getNamesAndAliases':{},
        'containsDirectMemberByNameOrAlias':{},
        'getTargetsWithRelationshipTypeForResource':{},
        'getReverseRelationshipsOfThisAndParents':{},
        'getPersonalResourceRoots':{},
        'getESMVersion':{},
        'getResourceByName':{},
        'hasXPermission':{},
        'findAllIds':{},
        'addRelationship':{},
        'getReverseRelationshipsOfParents':{},
        'getResourcesReferencePages':{},
        'containsDirectMemberByName':{},
        'getTargetsAsURIByRelationshipForSourceId':{},
        'hasWritePermission':{},
        'deleteResources':{},
        'resolveRelationship':{},
        'getAllPathsToRoot':{},
        'getResourcesWithVisibilityToUsers':{},
        'getReferencePages':{},
        'findAll':{},
        'getExclusivelyDependentResources':{},
        'getAllowedUserTypes':{},
        'getResourcesByNameSafely':{},
        'getResourcesByNames':{},
        'getSourceURIWithThisTargetByRelatiobnship':{},
        'getRelationshipsOfParents':{},
        'getMetaGroupID':{},
        'isValidResourceID':{},
        'getServiceMajorVersion':{},
        'hasReverseRelationship':{},
        'containsDirectMemberByName1':{},
        'getChildNamesAndAliases':{},
        'getResourcesByIds':{},
        'getResourceTypesVisibleToUsers':{},
        'update':{},
        'getRelationshipsOfThisAndParents':{},
        'getPersonalAndSharedResourceRoots':{},
        'getSourcesWithThisTargetByRelationship':{},
        'containsDirectMemberByNameOrAlias1':{},
        'hasReadPermission':{},
        'copyResourceIntoGroup':{},
        'deleteByLocalId':{},
        'getDependentResourceIDsForResourceId':{},
        'getResourceIfModified':{},
        'findById':{},
        'getSourcesWithThisTargetByRelationshipForResourceId':{},
        'getSourcesWithThisTargetByACLRelationship':{},
        'delete':{},
        'getAllPathsToRootAsStrings':{},
        'loadAdditional':{},
        'insertResource':{},
        'insertResources':{},
        'getAllUnassignedResourceIDs':{},
        'getEnabledResourceIDs':{},
        'getSourceURIWithThisTargetByRelationship':{},
        'getResourceById':{},
        'updateACLForResourceById':{},
        'getCorruptedResources':{},
        'unloadAdditional':{},
        'isDisabled':{},
        'getTargetsAsURIByRelationship':{},
        'updateResources':{},
        'deleteByUUID':{},
        'getTargetsByRelationship':{},
        'getSourceURIWithThisTargetByRelationshipForResourceId':{},
        'getTargetsByRelationshipCount':{},
        'getPersonalGroup':{},
        'findByUUID':{},
        'resetState':{},
        'insert':{},
        'getServiceMinorVersion':{},
    },
    'PortletService':{
        'getTargetsWithRelationshipTypeForResourceById':{},
        'getTargetsByRelationshipForSourceId':{},
        'getSourcesWithThisTargetByRelationshipCount':{},
        'deleteResource':{},
        'getSourceURIWithThisTargetByRelatiobnshipForResourceId':{},
        'getAllAttachmentOnlyResourceIDs':{},
        'getNamesAndAliases':{},
        'containsDirectMemberByNameOrAlias':{},
        'getTargetsWithRelationshipTypeForResource':{},
        'getReverseRelationshipsOfThisAndParents':{},
        'getPersonalResourceRoots':{},
        'getESMVersion':{},
        'getResourceByName':{},
        'hasXPermission':{},
        'findAllIds':{},
        'addRelationship':{},
        'getReverseRelationshipsOfParents':{},
        'getResourcesReferencePages':{},
        'containsDirectMemberByName':{},
        'getTargetsAsURIByRelationshipForSourceId':{},
        'hasWritePermission':{},
        'deleteResources':{},
        'resolveRelationship':{},
        'getAllPathsToRoot':{},
        'getResourcesWithVisibilityToUsers':{},
        'getReferencePages':{},
        'findAll':{},
        'getExclusivelyDependentResources':{},
        'getAllowedUserTypes':{},
        'getResourcesByNameSafely':{},
        'getResourcesByNames':{},
        'getSourceURIWithThisTargetByRelatiobnship':{},
        'getRelationshipsOfParents':{},
        'getMetaGroupID':{},
        'isValidResourceID':{},
        'getServiceMajorVersion':{},
        'hasReverseRelationship':{},
        'containsDirectMemberByName1':{},
        'getChildNamesAndAliases':{},
        'getResourcesByIds':{},
        'getResourceTypesVisibleToUsers':{},
        'update':{},
        'getRelationshipsOfThisAndParents':{},
        'getPersonalAndSharedResourceRoots':{},
        'getSourcesWithThisTargetByRelationship':{},
        'containsDirectMemberByNameOrAlias1':{},
        'hasReadPermission':{},
        'copyResourceIntoGroup':{},
        'deleteByLocalId':{},
        'getDependentResourceIDsForResourceId':{},
        'getResourceIfModified':{},
        'findById':{},
        'getSourcesWithThisTargetByRelationshipForResourceId':{},
        'getSourcesWithThisTargetByACLRelationship':{},
        'delete':{},
        'getAllPathsToRootAsStrings':{},
        'loadAdditional':{},
        'insertResource':{},
        'insertResources':{},
        'getAllUnassignedResourceIDs':{},
        'getEnabledResourceIDs':{},
        'getSourceURIWithThisTargetByRelationship':{},
        'getResourceById':{},
        'updateACLForResourceById':{},
        'getCorruptedResources':{},
        'unloadAdditional':{},
        'isDisabled':{},
        'getTargetsAsURIByRelationship':{},
        'updateResources':{},
        'deleteByUUID':{},
        'getTargetsByRelationship':{},
        'getSourceURIWithThisTargetByRelationshipForResourceId':{},
        'getTargetsByRelationshipCount':{},
        'getPersonalGroup':{},
        'findByUUID':{},
        'resetState':{},
        'insert':{},
        'getServiceMinorVersion':{},
    },
    'QueryService':{
        'getTargetsWithRelationshipTypeForResourceById':{},
        'getTargetsByRelationshipForSourceId':{},
        'getSourcesWithThisTargetByRelationshipCount':{},
        'deleteResource':{},
        'getSourceURIWithThisTargetByRelatiobnshipForResourceId':{},
        'getAllAttachmentOnlyResourceIDs':{},
        'getNamesAndAliases':{},
        'containsDirectMemberByNameOrAlias':{},
        'getTargetsWithRelationshipTypeForResource':{},
        'getReverseRelationshipsOfThisAndParents':{},
        'getPersonalResourceRoots':{},
        'getQuerySessionID':{},
        'getESMVersion':{},
        'getResourceByName':{},
        'hasXPermission':{},
        'findAllIds':{},
        'addRelationship':{},
        'getReverseRelationshipsOfParents':{},
        'getResourcesReferencePages':{},
        'containsDirectMemberByName':{},
        'getTargetsAsURIByRelationshipForSourceId':{},
        'hasWritePermission':{},
        'deleteResources':{},
        'resolveRelationship':{},
        'getQuerySQL':{},
        'getAllPathsToRoot':{},
        'getResourcesWithVisibilityToUsers':{},
        'getReferencePages':{},
        'findAll':{},
        'getExclusivelyDependentResources':{},
        'getAllowedUserTypes':{},
        'getResourcesByNameSafely':{},
        'getResourcesByNames':{},
        'getSourceURIWithThisTargetByRelatiobnship':{},
        'getRelationshipsOfParents':{},
        'getMetaGroupID':{},
        'isValidResourceID':{},
        'getServiceMajorVersion':{},
        'hasReverseRelationship':{},
        'containsDirectMemberByName1':{},
        'getChildNamesAndAliases':{},
        'getResourcesByIds':{},
        'getResourceTypesVisibleToUsers':{},
        'update':{},
        'getRelationshipsOfThisAndParents':{},
        'getPersonalAndSharedResourceRoots':{},
        'getSourcesWithThisTargetByRelationship':{},
        'containsDirectMemberByNameOrAlias1':{},
        'hasReadPermission':{},
        'copyResourceIntoGroup':{},
        'getDependentResourceIDsForResourceId':{},
        'deleteByLocalId':{},
        'getResourceIfModified':{},
        'findById':{},
        'getSourcesWithThisTargetByRelationshipForResourceId':{},
        'getSourcesWithThisTargetByACLRelationship':{},
        'delete':{},
        'getAllPathsToRootAsStrings':{},
        'loadAdditional':{},
        'insertResource':{},
        'insertResources':{},
        'getEnabledResourceIDs':{},
        'getAllUnassignedResourceIDs':{},
        'getSourceURIWithThisTargetByRelationship':{},
        'getResourceById':{},
        'updateACLForResourceById':{},
        'getCorruptedResources':{},
        'unloadAdditional':{},
        'isDisabled':{},
        'getTargetsAsURIByRelationship':{},
        'updateResources':{},
        'deleteByUUID':{},
        'getTargetsByRelationship':{},
        'getSourceURIWithThisTargetByRelationshipForResourceId':{},
        'getTargetsByRelationshipCount':{},
        'getPersonalGroup':{},
        'findByUUID':{},
        'resetState':{},
        'insert':{},
        'getServiceMinorVersion':{},
    },
    'ConnectorService':{  
        'getTargetsWithRelationshipTypeForResourceById':{},
        'getTargetsByRelationshipForSourceId':{},
        'getAllAttachmentOnlyResourceIDs':{},
        'getAllAgents':{},
        'getNamesAndAliases':{},
        'getReverseRelationshipsOfThisAndParents':{},
        'getAllRunningAgentIDs':{},
        'getESMVersion':{},
        'getAllAgentIDs':{},
        'addRelationship':{},
        'containsDirectMemberByName':{},
        'getAgentByName':{},
        'getAllStoppedAgentIDs':{},
        'getResourcesReferencePages':{},
        'getReverseRelationshipsOfParents':{},
        'getTargetsAsURIByRelationshipForSourceId':{},
        'sendCommand':{},
        'getReferencePages':{},
        'getExclusivelyDependentResources':{},
        'getAllowedUserTypes':{},
        'getResourcesByNames':{},
        'getSourceURIWithThisTargetByRelatiobnship':{},
        'getMetaGroupID':{},
        'isValidResourceID':{},
        'getServiceMajorVersion':{},
        'containsDirectMemberByName1':{},
        'getResourcesByIds':{},
        'getResourceTypesVisibleToUsers':{},
        'getConnectorExecStatus':{},
        'getPersonalAndSharedResourceRoots':{},
        'checkImportStatus':{},
        'hasReadPermission':{},
        'containsDirectMemberByNameOrAlias1':{},
        'getSourcesWithThisTargetByRelationship':{},
        'getDeadAgentIDs':{},
        'getAgentsByIDs':{},
        'findById':{},
        'getSourcesWithThisTargetByRelationshipForResourceId':{},
        'delete':{},
        'insertResource':{},
        'insertResources':{},
        'updateACLForResourceById':{},
        'getCorruptedResources':{},
        'unloadAdditional':{},
        'initiateImportConfiguration':{},
        'getTargetsByRelationship':{},
        'getPersonalGroup':{},
        'getTargetsByRelationshipCount':{},
        'getSourceURIWithThisTargetByRelationshipForResourceId':{},
        'getDevicesForAgents':{},
        'findByUUID':{},
        'resetState':{},
        'insert':{},
        'getAgentParameterDescriptor':{},
        'initiateExportConnectorConfiguration':{},
        'deleteResource':{},
        'getSourcesWithThisTargetByRelationshipCount':{},
        'getSourceURIWithThisTargetByRelatiobnshipForResourceId':{},
        'getAgentIDsByOperationalStatusType':{},
        'containsDirectMemberByNameOrAlias':{},
        'getTargetsWithRelationshipTypeForResource':{},
        'getPersonalResourceRoots':{},
        'getResourceByName':{},
        'hasXPermission':{},
        'findAllIds':{},
        'hasWritePermission':{},
        'deleteResources':{},
        'resolveRelationship':{},
        'getAllPathsToRoot':{},
        'getAgentParameterDescriptors':{},
        'getResourcesWithVisibilityToUsers':{},
        'findAll':{},
        'getResourcesByNameSafely':{},
        'getLiveAgentIDs':{},
        'getRelationshipsOfParents':{},
        'getAgentByID':{},
        'hasReverseRelationship':{},
        'getCommandsList':{},
        'getChildNamesAndAliases':{},
        'getParameterGroups':{},
        'update':{},
        'getAllPausedAgentIDs':{},
        'getRelationshipsOfThisAndParents':{},
        'updateConnector':{},
        'copyResourceIntoGroup':{},
        'deleteByLocalId':{},
        'getDependentResourceIDsForResourceId':{},
        'getResourceIfModified':{},
        'getSourcesWithThisTargetByACLRelationship':{},
        'getAllPathsToRootAsStrings':{},
        'executeCommand':{},
        'loadAdditional':{},
        'getAllUnassignedResourceIDs':{},
        'getEnabledResourceIDs':{},
        'getSourceURIWithThisTargetByRelationship':{},
        'getResourceById':{},
        'initiateDownloadFile':{},
        'isDisabled':{},
        'getTargetsAsURIByRelationship':{},
        'updateResources':{},
        'deleteByUUID':{},
        'getServiceMinorVersion':{},
    },
    'QueryViewerService':{
        'getTargetsWithRelationshipTypeForResourceById':{},
        'getTargetsByRelationshipForSourceId':{},
        'getSourcesWithThisTargetByRelationshipCount':{},
        'deleteResource':{},
        'getSourceURIWithThisTargetByRelatiobnshipForResourceId':{},
        'getAllAttachmentOnlyResourceIDs':{},
        'getNamesAndAliases':{},
        'containsDirectMemberByNameOrAlias':{},
        'getTargetsWithRelationshipTypeForResource':{},
        'getReverseRelationshipsOfThisAndParents':{},
        'getPersonalResourceRoots':{},
        'getESMVersion':{},
        'getResourceByName':{},
        'hasXPermission':{},
        'findAllIds':{},
        'addRelationship':{},
        'getReverseRelationshipsOfParents':{},
        'getResourcesReferencePages':{},
        'containsDirectMemberByName':{},
        'getTargetsAsURIByRelationshipForSourceId':{},
        'hasWritePermission':{},
        'deleteResources':{},
        'resolveRelationship':{},
        'getAllPathsToRoot':{},
        'getResourcesWithVisibilityToUsers':{},
        'getReferencePages':{},
        'getMatrixDataForDrilldown':{},
        'findAll':{},
        'getExclusivelyDependentResources':{},
        'getAllowedUserTypes':{},
        'getResourcesByNameSafely':{},
        'getResourcesByNames':{},
        'getSourceURIWithThisTargetByRelatiobnship':{},
        'getRelationshipsOfParents':{},
        'getMetaGroupID':{},
        'isValidResourceID':{},
        'getServiceMajorVersion':{},
        'hasReverseRelationship':{},
        'containsDirectMemberByName1':{},
        'getChildNamesAndAliases':{},
        'getResourcesByIds':{},
        'getResourceTypesVisibleToUsers':{},
        'update':{},
        'getRelationshipsOfThisAndParents':{},
        'getPersonalAndSharedResourceRoots':{},
        'getSourcesWithThisTargetByRelationship':{},
        'containsDirectMemberByNameOrAlias1':{},
        'hasReadPermission':{},
        'copyResourceIntoGroup':{},
        'getDependentResourceIDsForResourceId':{},
        'deleteByLocalId':{},
        'getResourceIfModified':{},
        'findById':{},
        'getSourcesWithThisTargetByRelationshipForResourceId':{},
        'getSourcesWithThisTargetByACLRelationship':{},
        'delete':{},
        'getAllPathsToRootAsStrings':{},
        'loadAdditional':{},
        'insertResource':{},
        'insertResources':{},
        'getEnabledResourceIDs':{},
        'getAllUnassignedResourceIDs':{},
        'getSourceURIWithThisTargetByRelationship':{},
        'getResourceById':{},
        'updateACLForResourceById':{},
        'getCorruptedResources':{},
        'unloadAdditional':{},
        'isDisabled':{},
        'getTargetsAsURIByRelationship':{},
        'updateResources':{},
        'deleteByUUID':{},
        'getTargetsByRelationship':{},
        'getSourceURIWithThisTargetByRelationshipForResourceId':{},
        'getMatrixData':{},
        'getTargetsByRelationshipCount':{},
        'getPersonalGroup':{},
        'findByUUID':{},
        'resetState':{},
        'insert':{},
        'getServiceMinorVersion':{},
    },
    'DrilldownListService':{
        'getTargetsWithRelationshipTypeForResourceById':{},
        'getTargetsByRelationshipForSourceId':{},
        'getSourcesWithThisTargetByRelationshipCount':{},
        'deleteResource':{},
        'getSourceURIWithThisTargetByRelatiobnshipForResourceId':{},
        'getAllAttachmentOnlyResourceIDs':{},
        'getNamesAndAliases':{},
        'containsDirectMemberByNameOrAlias':{},
        'getTargetsWithRelationshipTypeForResource':{},
        'getReverseRelationshipsOfThisAndParents':{},
        'getPersonalResourceRoots':{},
        'getESMVersion':{},
        'getResourceByName':{},
        'hasXPermission':{},
        'findAllIds':{},
        'addRelationship':{},
        'getReverseRelationshipsOfParents':{},
        'getResourcesReferencePages':{},
        'containsDirectMemberByName':{},
        'getDrilldownList':{},
        'getTargetsAsURIByRelationshipForSourceId':{},
        'hasWritePermission':{},
        'deleteResources':{},
        'resolveRelationship':{},
        'getAllPathsToRoot':{},
        'getResourcesWithVisibilityToUsers':{},
        'getReferencePages':{},
        'findAll':{},
        'getExclusivelyDependentResources':{},
        'getAllowedUserTypes':{},
        'getResourcesByNameSafely':{},
        'getResourcesByNames':{},
        'getSourceURIWithThisTargetByRelatiobnship':{},
        'getRelationshipsOfParents':{},
        'getMetaGroupID':{},
        'isValidResourceID':{},
        'getServiceMajorVersion':{},
        'hasReverseRelationship':{},
        'containsDirectMemberByName1':{},
        'getChildNamesAndAliases':{},
        'getResourcesByIds':{},
        'getResourceTypesVisibleToUsers':{},
        'update':{},
        'getRelationshipsOfThisAndParents':{},
        'getPersonalAndSharedResourceRoots':{},
        'getSourcesWithThisTargetByRelationship':{},
        'containsDirectMemberByNameOrAlias1':{},
        'hasReadPermission':{},
        'copyResourceIntoGroup':{},
        'deleteByLocalId':{},
        'getDependentResourceIDsForResourceId':{},
        'getResourceIfModified':{},
        'findById':{},
        'getSourcesWithThisTargetByRelationshipForResourceId':{},
        'getSourcesWithThisTargetByACLRelationship':{},
        'delete':{},
        'getAllPathsToRootAsStrings':{},
        'loadAdditional':{},
        'insertResource':{},
        'insertResources':{},
        'getEnabledResourceIDs':{},
        'getAllUnassignedResourceIDs':{},
        'getSourceURIWithThisTargetByRelationship':{},
        'getResourceById':{},
        'updateACLForResourceById':{},
        'getCorruptedResources':{},
        'unloadAdditional':{},
        'isDisabled':{},
        'getTargetsAsURIByRelationship':{},
        'updateResources':{},
        'deleteByUUID':{},
        'getTargetsByRelationship':{},
        'getSourceURIWithThisTargetByRelationshipForResourceId':{},
        'getTargetsByRelationshipCount':{},
        'getPersonalGroup':{},
        'findByUUID':{},
        'resetState':{},
        'insert':{},
        'getServiceMinorVersion':{},
    },
    'CaseService':{
        'getTargetsWithRelationshipTypeForResourceById':{},
        'getCaseEventIDs':{},
        'getTargetsByRelationshipForSourceId':{},
        'getSourcesWithThisTargetByRelationshipCount':{},
        'deleteResource':{},
        'getSourceURIWithThisTargetByRelatiobnshipForResourceId':{},
        'getAllAttachmentOnlyResourceIDs':{},
        'getNamesAndAliases':{},
        'containsDirectMemberByNameOrAlias':{},
        'getTargetsWithRelationshipTypeForResource':{},
        'getReverseRelationshipsOfThisAndParents':{},
        'getPersonalResourceRoots':{},
        'getESMVersion':{},
        'getResourceByName':{},
        'hasXPermission':{},
        'findAllIds':{},
        'addRelationship':{},
        'getCasesGroupID':{},
        'getReverseRelationshipsOfParents':{},
        'getResourcesReferencePages':{},
        'containsDirectMemberByName':{},
        'getTargetsAsURIByRelationshipForSourceId':{},
        'hasWritePermission':{},
        'deleteResources':{},
        'resolveRelationship':{},
        'getAllPathsToRoot':{},
        'getResourcesWithVisibilityToUsers':{},
        'getReferencePages':{},
        'findAll':{},
        'getExclusivelyDependentResources':{},
        'getAllowedUserTypes':{},
        'getResourcesByNameSafely':{},
        'getResourcesByNames':{},
        'getSourceURIWithThisTargetByRelatiobnship':{},
        'getRelationshipsOfParents':{},
        'getMetaGroupID':{},
        'isValidResourceID':{},
        'getServiceMajorVersion':{},
        'hasReverseRelationship':{},
        'getChildNamesAndAliases':{},
        'containsDirectMemberByName1':{},
        'getResourcesByIds':{},
        'getResourceTypesVisibleToUsers':{},
        'update':{},
        'getRelationshipsOfThisAndParents':{},
        'getPersonalAndSharedResourceRoots':{},
        'getSourcesWithThisTargetByRelationship':{},
        'containsDirectMemberByNameOrAlias1':{},
        'hasReadPermission':{},
        'copyResourceIntoGroup':{},
        'getDependentResourceIDsForResourceId':{},
        'deleteByLocalId':{},
        'getResourceIfModified':{},
        'findById':{},
        'getSourcesWithThisTargetByRelationshipForResourceId':{},
        'getSourcesWithThisTargetByACLRelationship':{},
        'delete':{},
        'getAllPathsToRootAsStrings':{},
        'loadAdditional':{},
        'deleteAllCaseEvents':{},
        'insertResource':{},
        'insertResources':{},
        'addCaseEvents':{},
        'getEnabledResourceIDs':{},
        'getAllUnassignedResourceIDs':{},
        'getSourceURIWithThisTargetByRelationship':{},
        'getResourceById':{},
        'updateACLForResourceById':{},
        'getCorruptedResources':{},
        'deleteCaseEvents':{},
        'unloadAdditional':{},
        'isDisabled':{},
        'getTargetsAsURIByRelationship':{},
        'updateResources':{},
        'deleteByUUID':{},
        'getTargetsByRelationship':{},
        'getSourceURIWithThisTargetByRelationshipForResourceId':{},
        'getTargetsByRelationshipCount':{},
        'getPersonalGroup':{},
        'findByUUID':{},
        'resetState':{},
        'getCaseEventsTimeSpan':{},
        'getSystemCasesGroupID':{},
        'insert':{},
        'getServiceMinorVersion':{},
        'getEventExportStatus':{},
    },
    'ArchiveReportService':{
        'getTargetsWithRelationshipTypeForResourceById':{},
        'getTargetsByRelationshipForSourceId':{},
        'getSourcesWithThisTargetByRelationshipCount':{},
        'deleteResource':{},
        'getSourceURIWithThisTargetByRelatiobnshipForResourceId':{},
        'getAllAttachmentOnlyResourceIDs':{},
        'getNamesAndAliases':{},
        'containsDirectMemberByNameOrAlias':{},
        'getTargetsWithRelationshipTypeForResource':{},
        'getReverseRelationshipsOfThisAndParents':{},
        'getPersonalResourceRoots':{},
        'getDefaultArchiveReportByURI':{},
        'getESMVersion':{},
        'getResourceByName':{},
        'hasXPermission':{},
        'poll':{},
        'findAllIds':{},
        'addRelationship':{},
        'getReverseRelationshipsOfParents':{},
        'getResourcesReferencePages':{},
        'containsDirectMemberByName':{},
        'getTargetsAsURIByRelationshipForSourceId':{},
        'hasWritePermission':{},
        'deleteResources':{},
        'getAllPathsToRoot':{},
        'resolveRelationship':{},
        'getResourcesWithVisibilityToUsers':{},
        'getReferencePages':{},
        'findAll':{},
        'getExclusivelyDependentResources':{},
        'getAllowedUserTypes':{},
        'initDefaultArchiveReportDownloadWithOverwrite':{},
        'getResourcesByNameSafely':{},
        'getResourcesByNames':{},
        'getSourceURIWithThisTargetByRelatiobnship':{},
        'getDefaultArchiveReportById':{},
        'getRelationshipsOfParents':{},
        'getMetaGroupID':{},
        'isValidResourceID':{},
        'archiveReport':{},
        'getServiceMajorVersion':{},
        'initDefaultArchiveReportDownloadByURI':{},
        'hasReverseRelationship':{},
        'getChildNamesAndAliases':{},
        'containsDirectMemberByName1':{},
        'getResourcesByIds':{},
        'getResourceTypesVisibleToUsers':{},
        'update':{},
        'getRelationshipsOfThisAndParents':{},
        'getPersonalAndSharedResourceRoots':{},
        'getSourcesWithThisTargetByRelationship':{},
        'containsDirectMemberByNameOrAlias1':{},
        'hasReadPermission':{},
        'copyResourceIntoGroup':{},
        'getDependentResourceIDsForResourceId':{},
        'deleteByLocalId':{},
        'initDefaultArchiveReportDownloadById':{},
        'getResourceIfModified':{},
        'findById':{},
        'getSourcesWithThisTargetByRelationshipForResourceId':{},
        'getSourcesWithThisTargetByACLRelationship':{},
        'delete':{},
        'getAllPathsToRootAsStrings':{},
        'loadAdditional':{},
        'insertResource':{},
        'insertResources':{},
        'getEnabledResourceIDs':{},
        'getAllUnassignedResourceIDs':{},
        'getSourceURIWithThisTargetByRelationship':{},
        'getResourceById':{},
        'initDefaultArchiveReportDownloadByIdASync':{},
        'updateACLForResourceById':{},
        'getCorruptedResources':{},
        'unloadAdditional':{},
        'isDisabled':{},
        'getTargetsAsURIByRelationship':{},
        'updateResources':{},
        'deleteByUUID':{},
        'getTargetsByRelationship':{},
        'getSourceURIWithThisTargetByRelationshipForResourceId':{},
        'getTargetsByRelationshipCount':{},
        'getPersonalGroup':{},
        'findByUUID':{},
        'resetState':{},
        'insert':{},
        'getServiceMinorVersion':{},
    },
    'ActiveListService':{
        'getTargetsWithRelationshipTypeForResourceById':{},
        'getTargetsByRelationshipForSourceId':{},
        'getSourcesWithThisTargetByRelationshipCount':{},
        'deleteResource':{},
        'getSourceURIWithThisTargetByRelatiobnshipForResourceId':{},
        'getAllAttachmentOnlyResourceIDs':{},
        'getNamesAndAliases':{},
        'containsDirectMemberByNameOrAlias':{},
        'getTargetsWithRelationshipTypeForResource':{},
        'getReverseRelationshipsOfThisAndParents':{},
        'getPersonalResourceRoots':{},
        'getESMVersion':{},
        'getResourceByName':{},
        'hasXPermission':{},
        'addEntries':{},
        'findAllIds':{},
        'addRelationship':{},
        'getReverseRelationshipsOfParents':{},
        'getResourcesReferencePages':{},
        'containsDirectMemberByName':{},
        'getTargetsAsURIByRelationshipForSourceId':{},
        'hasWritePermission':{},
        'deleteResources':{},
        'resolveRelationship':{},
        'getAllPathsToRoot':{},
        'getResourcesWithVisibilityToUsers':{},
        'getReferencePages':{},
        'findAll':{},
        'getExclusivelyDependentResources':{},
        'getAllowedUserTypes':{},
        'getResourcesByNameSafely':{},
        'getResourcesByNames':{},
        'getSourceURIWithThisTargetByRelatiobnship':{},
        'getRelationshipsOfParents':{},
        'getMetaGroupID':{},
        'isValidResourceID':{},
        'getServiceMajorVersion':{},
        'getEntries':{},
        'hasReverseRelationship':{},
        'getChildNamesAndAliases':{},
        'containsDirectMemberByName1':{},
        'getResourcesByIds':{},
        'getResourceTypesVisibleToUsers':{},
        'update':{},
        'getRelationshipsOfThisAndParents':{},
        'getPersonalAndSharedResourceRoots':{},
        'getSourcesWithThisTargetByRelationship':{},
        'containsDirectMemberByNameOrAlias1':{},
        'hasReadPermission':{},
        'copyResourceIntoGroup':{},
        'getDependentResourceIDsForResourceId':{},
        'deleteByLocalId':{},
        'clearEntries':{},
        'getResourceIfModified':{},
        'findById':{},
        'getSourcesWithThisTargetByRelationshipForResourceId':{},
        'getSourcesWithThisTargetByACLRelationship':{},
        'delete':{},
        'getAllPathsToRootAsStrings':{},
        'loadAdditional':{},
        'deleteEntries':{},
        'insertResource':{},
        'insertResources':{},
        'getEnabledResourceIDs':{},
        'getAllUnassignedResourceIDs':{},
        'getSourceURIWithThisTargetByRelationship':{},
        'getResourceById':{},
        'updateACLForResourceById':{},
        'getCorruptedResources':{},
        'unloadAdditional':{},
        'isDisabled':{},
        'getTargetsAsURIByRelationship':{},
        'updateResources':{},
        'deleteByUUID':{},
        'getTargetsByRelationship':{},
        'getSourceURIWithThisTargetByRelationshipForResourceId':{},
        'getTargetsByRelationshipCount':{},
        'getPersonalGroup':{},
        'findByUUID':{},
        'resetState':{},
        'insert':{},
        'getServiceMinorVersion':{},
    },
    'InternalService':{
        'newGroupAttributesParameter':{},
        'newReportDrilldownDefinition':{},
        'newEdge':{},
        'newHierarchyMapGroupByHolder':{},
        'newActiveChannelDrilldownDefinition':{},
        'newMatrixData':{},
        'newIntrospectableFieldListParameter':{},
        'newGraphData':{},
        'newGeoInfoEventGraphNode':{},
        'newIntrospectableFieldListHolder':{},
        'newGroupAttributeEntry':{},
        'newDashboardDrilldownDefinition':{},
        'newListWrapper':{},
        'newFontHolder':{},
        'newEventGraph':{},
        'newPropertyHolder':{},
        'newQueryViewerDrilldownDefinition':{},
        'newGraph':{},
        'newFilterFields':{},
        'newFontParameter':{},
        'newGeographicInformation':{},
        'newNode':{},
        'newHierarchyMapGroupByParameter':{},
        'newErrorCode':{},
        'newEventGraphNode':{},
    },
    'SecurityEventService':{
        'getServiceMajorVersion':{},
        'getSecurityEventsWithTimeout':{},
        'getSecurityEvents':{},
        'getServiceMinorVersion':{},
        'getSecurityEventsByProfile':{},
    },
    'GraphService':{
        'createSourceTargetGraphFromEventList':{},
        'createSourceTargetGraph':{},
        'createSourceEventTargetGraph':{},
        'getServiceMajorVersion':{},
        'getServiceMinorVersion':{},
        'createSourceEventTargetGraphFromEventList':{},
    },
    'GroupService':{
        'getTargetsWithRelationshipTypeForResourceById':{},
        'childAttributesChanged':{},
        'getTargetsByRelationshipForSourceId':{},
        'getAllAttachmentOnlyResourceIDs':{},
        'getNamesAndAliases':{},
        'getReverseRelationshipsOfThisAndParents':{},
        'getESMVersion':{},
        'addRelationship':{},
        'containsDirectMemberByName':{},
        'getResourcesReferencePages':{},
        'getReverseRelationshipsOfParents':{},
        'getTargetsAsURIByRelationshipForSourceId':{},
        'isParentOf':{},
        'insertGroup':{},
        'getAllChildren':{},
        'getReferencePages':{},
        'getGroupChildCount':{},
        'getExclusivelyDependentResources':{},
        'getAllowedUserTypes':{},
        'getResourcesByNames':{},
        'removeChild':{},
        'getSourceURIWithThisTargetByRelatiobnship':{},
        'getMetaGroupID':{},
        'isValidResourceID':{},
        'addChild':{},
        'getServiceMajorVersion':{},
        'containsDirectMemberByName1':{},
        'getResourcesByIds':{},
        'getResourceTypesVisibleToUsers':{},
        'getPersonalAndSharedResourceRoots':{},
        'hasReadPermission':{},
        'containsDirectMemberByNameOrAlias1':{},
        'getSourcesWithThisTargetByRelationship':{},
        'findById':{},
        'getSourcesWithThisTargetByRelationshipForResourceId':{},
        'delete':{},
        'insertResource':{},
        'insertResources':{},
        'getAllChildIDCount':{},
        'updateACLForResourceById':{},
        'getCorruptedResources':{},
        'unloadAdditional':{},
        'getTargetsByRelationship':{},
        'getPersonalGroup':{},
        'getTargetsByRelationshipCount':{},
        'getSourceURIWithThisTargetByRelationshipForResourceId':{},
        'findByUUID':{},
        'resetState':{},
        'insert':{},
        'deleteResource':{},
        'getSourcesWithThisTargetByRelationshipCount':{},
        'getSourceURIWithThisTargetByRelatiobnshipForResourceId':{},
        'containsDirectMemberByNameOrAlias':{},
        'removeChildren':{},
        'getTargetsWithRelationshipTypeForResource':{},
        'getPersonalResourceRoots':{},
        'getMetaGroup':{},
        'getChildrenByType':{},
        'getResourceByName':{},
        'hasXPermission':{},
        'findAllIds':{},
        'hasWritePermission':{},
        'deleteResources':{},
        'resolveRelationship':{},
        'getAllPathsToRoot':{},
        'getResourcesWithVisibilityToUsers':{},
        'findAll':{},
        'addChildren':{},
        'getResourcesByNameSafely':{},
        'getRelationshipsOfParents':{},
        'getChildResourcesByType':{},
        'hasReverseRelationship':{},
        'getChildNamesAndAliases':{},
        'update':{},
        'hasChildWithNameOrAlias':{},
        'getRelationshipsOfThisAndParents':{},
        'getChildIDByChildNameOrAlias':{},
        'containsResourcesRecursively':{},
        'copyResourceIntoGroup':{},
        'deleteByLocalId':{},
        'getDependentResourceIDsForResourceId':{},
        'getResourceIfModified':{},
        'isGroup':{},
        'getSourcesWithThisTargetByACLRelationship':{},
        'getAllPathsToRootAsStrings':{},
        'updateGroup':{},
        'loadAdditional':{},
        'getAllUnassignedResourceIDs':{},
        'getEnabledResourceIDs':{},
        'getSourceURIWithThisTargetByRelationship':{},
        'getResourceById':{},
        'getGroupByURI':{},
        'isDisabled':{},
        'getAllChildIDs':{},
        'getTargetsAsURIByRelationship':{},
        'updateResources':{},
        'deleteByUUID':{},
        'getGroupByID':{},
        'getServiceMinorVersion':{},
    },
    'ResourceService':{
        'getTargetsWithRelationshipTypeForResourceById':{},
        'getTargetsByRelationshipForSourceId':{},
        'getSourcesWithThisTargetByRelationshipCount':{},
        'deleteResource':{},
        'getSourceURIWithThisTargetByRelatiobnshipForResourceId':{},
        'getAllAttachmentOnlyResourceIDs':{},
        'getNamesAndAliases':{},
        'containsDirectMemberByNameOrAlias':{},
        'getTargetsWithRelationshipTypeForResource':{},
        'getReverseRelationshipsOfThisAndParents':{},
        'getPersonalResourceRoots':{},
        'getESMVersion':{},
        'getResourceByName':{},
        'hasXPermission':{},
        'findAllIds':{},
        'addRelationship':{},
        'getReverseRelationshipsOfParents':{},
        'getResourcesReferencePages':{},
        'containsDirectMemberByName':{},
        'getTargetsAsURIByRelationshipForSourceId':{},
        'hasWritePermission':{},
        'deleteResources':{},
        'resolveRelationship':{},
        'getAllPathsToRoot':{},
        'getResourcesWithVisibilityToUsers':{},
        'getReferencePages':{},
        'findAll':{},
        'getExclusivelyDependentResources':{},
        'getAllowedUserTypes':{},
        'getResourcesByNameSafely':{},
        'getResourcesByNames':{},
        'getSourceURIWithThisTargetByRelatiobnship':{},
        'getRelationshipsOfParents':{},
        'getMetaGroupID':{},
        'isValidResourceID':{},
        'getServiceMajorVersion':{},
        'hasReverseRelationship':{},
        'containsDirectMemberByName1':{},
        'getChildNamesAndAliases':{},
        'getResourcesByIds':{},
        'getResourceTypesVisibleToUsers':{},
        'update':{},
        'getRelationshipsOfThisAndParents':{},
        'getPersonalAndSharedResourceRoots':{},
        'getSourcesWithThisTargetByRelationship':{},
        'containsDirectMemberByNameOrAlias1':{},
        'hasReadPermission':{},
        'copyResourceIntoGroup':{},
        'deleteByLocalId':{},
        'getDependentResourceIDsForResourceId':{},
        'getResourceIfModified':{},
        'findById':{},
        'getSourcesWithThisTargetByRelationshipForResourceId':{},
        'getSourcesWithThisTargetByACLRelationship':{},
        'delete':{},
        'getAllPathsToRootAsStrings':{},
        'loadAdditional':{},
        'insertResource':{},
        'insertResources':{},
        'getAllUnassignedResourceIDs':{},
        'getEnabledResourceIDs':{},
        'getSourceURIWithThisTargetByRelationship':{},
        'getResourceById':{},
        'updateACLForResourceById':{},
        'getCorruptedResources':{},
        'unloadAdditional':{},
        'isDisabled':{},
        'getTargetsAsURIByRelationship':{},
        'updateResources':{},
        'deleteByUUID':{},
        'getTargetsByRelationship':{},
        'getSourceURIWithThisTargetByRelationshipForResourceId':{},
        'getTargetsByRelationshipCount':{},
        'getPersonalGroup':{},
        'findByUUID':{},
        'resetState':{},
        'insert':{},
        'getServiceMinorVersion':{},
    },
    'UserResourceService':{
        'getTargetsWithRelationshipTypeForResourceById':{},
        'getTargetsByRelationshipForSourceId':{},
        'getAllAttachmentOnlyResourceIDs':{},
        'addUserPreferenceById':{},
        'getNamesAndAliases':{},
        'getReverseRelationshipsOfThisAndParents':{},
        'getESMVersion':{},
        'addRelationship':{},
        'containsDirectMemberByName':{},
        'getResourcesReferencePages':{},
        'getReverseRelationshipsOfParents':{},
        'getTargetsAsURIByRelationshipForSourceId':{},
        'getReferencePages':{},
        'getUserModificationFlag':{},
        'getAllUsers':{},
        'getExclusivelyDependentResources':{},
        'getAllowedUserTypes':{},
        'create':{},
        'getResourcesByNames':{},
        'getSourceURIWithThisTargetByRelatiobnship':{},
        'recordSuccessfulLoginFor':{},
        'updateUserPreferencesByName':{},
        'getMetaGroupID':{},
        'isValidResourceID':{},
        'getServiceMajorVersion':{},
        'changePassword':{},
        'containsDirectMemberByName1':{},
        'getResourcesByIds':{},
        'getResourceTypesVisibleToUsers':{},
        'getCurrentUser':{},
        'getPersonalAndSharedResourceRoots':{},
        'getSourcesWithThisTargetByRelationship':{},
        'hasReadPermission':{},
        'containsDirectMemberByNameOrAlias1':{},
        'getUserByName':{},
        'getSessionProfile':{},
        'findById':{},
        'getSourcesWithThisTargetByRelationshipForResourceId':{},
        'delete':{},
        'getRootUserGroup':{},
        'updateUserPreferencesById':{},
        'getAllUserPreferencesForUserByName':{},
        'insertResource':{},
        'insertResources':{},
        'updateACLForResourceById':{},
        'getCorruptedResources':{},
        'unloadAdditional':{},
        'getFeatureAvailabilities':{},
        'isFeatureAvailable':{},
        'getTargetsByRelationship':{},
        'increaseFailedLoginAttemptsFor':{},
        'getSourceURIWithThisTargetByRelationshipForResourceId':{},
        'getPersonalGroup':{},
        'getTargetsByRelationshipCount':{},
        'findByUUID':{},
        'resetState':{},
        'insert':{},
        'isAdministrator':{},
        'getSourcesWithThisTargetByRelationshipCount':{},
        'deleteResource':{},
        'getSourceURIWithThisTargetByRelatiobnshipForResourceId':{},
        'containsDirectMemberByNameOrAlias':{},
        'getTargetsWithRelationshipTypeForResource':{},
        'getPersonalResourceRoots':{},
        'getResourceByName':{},
        'hasXPermission':{},
        'addModuleConfigForUserById':{},
        'findAllIds':{},
        'hasWritePermission':{},
        'deleteResources':{},
        'resolveRelationship':{},
        'getAllPathsToRoot':{},
        'getUserPreferenceForUserByName':{},
        'getAllUserPreferencesForUserById':{},
        'getModuleConfigForUserByName':{},
        'getResourcesWithVisibilityToUsers':{},
        'getUserPreferenceForUserById':{},
        'findAll':{},
        'getResourcesByNameSafely':{},
        'getRelationshipsOfParents':{},
        'getServerDefaultLocale':{},
        'hasReverseRelationship':{},
        'getChildNamesAndAliases':{},
        'updateUser':{},
        'update':{},
        'updateModuleConfigForUserById':{},
        'getRelationshipsOfThisAndParents':{},
        'copyResourceIntoGroup':{},
        'deleteByLocalId':{},
        'getDependentResourceIDsForResourceId':{},
        'checkPassword':{},
        'updateModuleConfigForUserByName':{},
        'getResourceIfModified':{},
        'addModuleConfigForUserByName':{},
        'getSourcesWithThisTargetByACLRelationship':{},
        'getAllPathsToRootAsStrings':{},
        'getRootUserGroupID':{},
        'loadAdditional':{},
        'resetFailedLoginAttemptsFor':{},
        'getAllUnassignedResourceIDs':{},
        'getEnabledResourceIDs':{},
        'getSourceURIWithThisTargetByRelationship':{},
        'getResourceById':{},
        'isDisabled':{},
        'getTargetsAsURIByRelationship':{},
        'updateResources':{},
        'deleteByUUID':{},
        'getRootUserId':{},
        'getModuleConfigForUserById':{},
        'addUserPreferenceByName':{},
        'getServiceMinorVersion':{},
    },
    'FileResourceService':{
        'getTargetsWithRelationshipTypeForResourceById':{},
        'getTargetsByRelationshipForSourceId':{},
        'initiateUpload':{},
        'getSourcesWithThisTargetByRelationshipCount':{},
        'initiateDownloadByUUID':{},
        'deleteResource':{},
        'getSourceURIWithThisTargetByRelatiobnshipForResourceId':{},
        'getAllAttachmentOnlyResourceIDs':{},
        'getNamesAndAliases':{},
        'containsDirectMemberByNameOrAlias':{},
        'getTargetsWithRelationshipTypeForResource':{},
        'getReverseRelationshipsOfThisAndParents':{},
        'getPersonalResourceRoots':{},
        'getESMVersion':{},
        'getResourceByName':{},
        'hasXPermission':{},
        'findAllIds':{},
        'addRelationship':{},
        'getReverseRelationshipsOfParents':{},
        'getResourcesReferencePages':{},
        'containsDirectMemberByName':{},
        'getTargetsAsURIByRelationshipForSourceId':{},
        'hasWritePermission':{},
        'deleteResources':{},
        'resolveRelationship':{},
        'getAllPathsToRoot':{},
        'getResourcesWithVisibilityToUsers':{},
        'getReferencePages':{},
        'findAll':{},
        'getUploadStatus':{},
        'getExclusivelyDependentResources':{},
        'getAllowedUserTypes':{},
        'getResourcesByNameSafely':{},
        'getResourcesByNames':{},
        'getSourceURIWithThisTargetByRelatiobnship':{},
        'getRelationshipsOfParents':{},
        'getMetaGroupID':{},
        'isValidResourceID':{},
        'getServiceMajorVersion':{},
        'hasReverseRelationship':{},
        'containsDirectMemberByName1':{},
        'getChildNamesAndAliases':{},
        'getResourcesByIds':{},
        'getResourceTypesVisibleToUsers':{},
        'update':{},
        'getRelationshipsOfThisAndParents':{},
        'getPersonalAndSharedResourceRoots':{},
        'getSourcesWithThisTargetByRelationship':{},
        'containsDirectMemberByNameOrAlias1':{},
        'hasReadPermission':{},
        'copyResourceIntoGroup':{},
        'getDependentResourceIDsForResourceId':{},
        'deleteByLocalId':{},
        'getResourceIfModified':{},
        'findById':{},
        'getSourcesWithThisTargetByRelationshipForResourceId':{},
        'getSourcesWithThisTargetByACLRelationship':{},
        'delete':{},
        'getAllPathsToRootAsStrings':{},
        'loadAdditional':{},
        'insertResource':{},
        'insertResources':{},
        'getEnabledResourceIDs':{},
        'getAllUnassignedResourceIDs':{},
        'getSourceURIWithThisTargetByRelationship':{},
        'getResourceById':{},
        'updateACLForResourceById':{},
        'getCorruptedResources':{},
        'unloadAdditional':{},
        'isDisabled':{},
        'getTargetsAsURIByRelationship':{},
        'updateResources':{},
        'deleteByUUID':{},
        'getTargetsByRelationship':{},
        'getSourceURIWithThisTargetByRelationshipForResourceId':{},
        'getTargetsByRelationshipCount':{},
        'getPersonalGroup':{},
        'findByUUID':{},
        'resetState':{},
        'insert':{},
        'getServiceMinorVersion':{},
    },
    'InfoService':{
        'getActivationDateMillis':{},
        'getPropertyByEncodedKey':{},
        'isTrial':{},
        'hasErrors':{},
        'getWebServerUrl':{},
        'getWebAdminRelUrlWithOTP':{},
        'getWebAdminRelUrl':{},
        'isPatternDiscoveryEnabled':{},
        'getExpirationDate':{},
        'getWebServerUrlWithOTP':{},
        'isLicenseValid':{},
        'getErrorMessage':{},
        'getManagerVersionString':{},
        'expires':{},
        'isSessionListsEnabled':{},
        'setLicensed':{},
        'getProperty':{},
        'isPartitionArchiveEnabled':{},
        'getStatusString':{},
        'getServiceMajorVersion':{},
        'getCustomerName':{},
        'getCustomerNumber':{},
        'getServiceMinorVersion':{},
    },
    'SecurityEventIntrospectorService':{
        'getTimeConstraintFields':{},
        'hasField':{},
        'convertLabelToName':{},
        'getFields':{},
        'getGroupNames':{},
        'getServiceMajorVersion':{},
        'getFieldsByFilter':{},
        'hasFieldName':{},
        'getFieldByName':{},
        'getGroupDisplayName':{},
        'getServiceMinorVersion':{},
        'getRelatedFields':{},
    },
    'ConAppService':{
        'getPathToConApp':{},
        'getServiceMajorVersion':{},
        'getServiceMinorVersion':{},
    },
    'FieldSetService':{
        'getTargetsWithRelationshipTypeForResourceById':{},
        'getTargetsByRelationshipForSourceId':{},
        'getSourcesWithThisTargetByRelationshipCount':{},
        'deleteResource':{},
        'getSourceURIWithThisTargetByRelatiobnshipForResourceId':{},
        'getAllAttachmentOnlyResourceIDs':{},
        'getNamesAndAliases':{},
        'containsDirectMemberByNameOrAlias':{},
        'getTargetsWithRelationshipTypeForResource':{},
        'getReverseRelationshipsOfThisAndParents':{},
        'getPersonalResourceRoots':{},
        'getESMVersion':{},
        'getResourceByName':{},
        'hasXPermission':{},
        'findAllIds':{},
        'addRelationship':{},
        'getReverseRelationshipsOfParents':{},
        'getResourcesReferencePages':{},
        'containsDirectMemberByName':{},
        'getTargetsAsURIByRelationshipForSourceId':{},
        'hasWritePermission':{},
        'deleteResources':{},
        'resolveRelationship':{},
        'getAllPathsToRoot':{},
        'getResourcesWithVisibilityToUsers':{},
        'getReferencePages':{},
        'findAll':{},
        'getExclusivelyDependentResources':{},
        'getAllowedUserTypes':{},
        'getResourcesByNameSafely':{},
        'getResourcesByNames':{},
        'getSourceURIWithThisTargetByRelatiobnship':{},
        'getRelationshipsOfParents':{},
        'getMetaGroupID':{},
        'isValidResourceID':{},
        'getServiceMajorVersion':{},
        'hasReverseRelationship':{},
        'containsDirectMemberByName1':{},
        'getChildNamesAndAliases':{},
        'getResourcesByIds':{},
        'getResourceTypesVisibleToUsers':{},
        'update':{},
        'getRelationshipsOfThisAndParents':{},
        'getPersonalAndSharedResourceRoots':{},
        'getSourcesWithThisTargetByRelationship':{},
        'containsDirectMemberByNameOrAlias1':{},
        'hasReadPermission':{},
        'copyResourceIntoGroup':{},
        'deleteByLocalId':{},
        'getDependentResourceIDsForResourceId':{},
        'getResourceIfModified':{},
        'findById':{},
        'getSourcesWithThisTargetByRelationshipForResourceId':{},
        'getSourcesWithThisTargetByACLRelationship':{},
        'delete':{},
        'getAllPathsToRootAsStrings':{},
        'loadAdditional':{},
        'insertResource':{},
        'insertResources':{},
        'getAllUnassignedResourceIDs':{},
        'getEnabledResourceIDs':{},
        'getSourceURIWithThisTargetByRelationship':{},
        'getResourceById':{},
        'updateACLForResourceById':{},
        'getCorruptedResources':{},
        'unloadAdditional':{},
        'isDisabled':{},
        'getTargetsAsURIByRelationship':{},
        'updateResources':{},
        'deleteByUUID':{},
        'getTargetsByRelationship':{},
        'getSourceURIWithThisTargetByRelationshipForResourceId':{},
        'getTargetsByRelationshipCount':{},
        'getPersonalGroup':{},
        'findByUUID':{},
        'resetState':{},
        'insert':{},
        'getServiceMinorVersion':{},
    },
    'ReportService':{
        'getTargetsWithRelationshipTypeForResourceById':{},
        'getTargetsByRelationshipForSourceId':{},
        'getSourcesWithThisTargetByRelationshipCount':{},
        'deleteResource':{},
        'getSourceURIWithThisTargetByRelatiobnshipForResourceId':{},
        'getAllAttachmentOnlyResourceIDs':{},
        'getNamesAndAliases':{},
        'containsDirectMemberByNameOrAlias':{},
        'getTargetsWithRelationshipTypeForResource':{},
        'getReverseRelationshipsOfThisAndParents':{},
        'getPersonalResourceRoots':{},
        'getESMVersion':{},
        'getResourceByName':{},
        'hasXPermission':{},
        'findAllIds':{},
        'addRelationship':{},
        'getReverseRelationshipsOfParents':{},
        'getResourcesReferencePages':{},
        'containsDirectMemberByName':{},
        'getTargetsAsURIByRelationshipForSourceId':{},
        'hasWritePermission':{},
        'deleteResources':{},
        'resolveRelationship':{},
        'getAllPathsToRoot':{},
        'getResourcesWithVisibilityToUsers':{},
        'getReferencePages':{},
        'findAll':{},
        'getExclusivelyDependentResources':{},
        'getAllowedUserTypes':{},
        'getResourcesByNameSafely':{},
        'getResourcesByNames':{},
        'getSourceURIWithThisTargetByRelatiobnship':{},
        'getRelationshipsOfParents':{},
        'getMetaGroupID':{},
        'isValidResourceID':{},
        'getServiceMajorVersion':{},
        'hasReverseRelationship':{},
        'containsDirectMemberByName1':{},
        'getChildNamesAndAliases':{},
        'getResourcesByIds':{},
        'getResourceTypesVisibleToUsers':{},
        'update':{},
        'getRelationshipsOfThisAndParents':{},
        'getPersonalAndSharedResourceRoots':{},
        'getSourcesWithThisTargetByRelationship':{},
        'containsDirectMemberByNameOrAlias1':{},
        'hasReadPermission':{},
        'copyResourceIntoGroup':{},
        'deleteByLocalId':{},
        'getDependentResourceIDsForResourceId':{},
        'getResourceIfModified':{},
        'findById':{},
        'getSourcesWithThisTargetByRelationshipForResourceId':{},
        'getSourcesWithThisTargetByACLRelationship':{},
        'delete':{},
        'getAllPathsToRootAsStrings':{},
        'loadAdditional':{},
        'insertResource':{},
        'insertResources':{},
        'getAllUnassignedResourceIDs':{},
        'getEnabledResourceIDs':{},
        'getSourceURIWithThisTargetByRelationship':{},
        'getResourceById':{},
        'updateACLForResourceById':{},
        'getCorruptedResources':{},
        'unloadAdditional':{},
        'isDisabled':{},
        'getTargetsAsURIByRelationship':{},
        'updateResources':{},
        'deleteByUUID':{},
        'getTargetsByRelationship':{},
        'getSourceURIWithThisTargetByRelationshipForResourceId':{},
        'getTargetsByRelationshipCount':{},
        'getPersonalGroup':{},
        'findByUUID':{},
        'resetState':{},
        'insert':{},
        'getServiceMinorVersion':{},
    },
    'ManagerAuthenticationService':{
        'getServiceMajorVersion':{},
        'getOTP':{},
        'getServiceMinorVersion':{},
    },
    'ManagerSearchService':{
        'search':{},
        'search1':{},
    },
    'DataMonitorService':{
        'getTargetsWithRelationshipTypeForResourceById':{},
        'getTargetsByRelationshipForSourceId':{},
        'getSourcesWithThisTargetByRelationshipCount':{},
        'deleteResource':{},
        'getSourceURIWithThisTargetByRelatiobnshipForResourceId':{},
        'getAllAttachmentOnlyResourceIDs':{},
        'getNamesAndAliases':{},
        'containsDirectMemberByNameOrAlias':{},
        'getTargetsWithRelationshipTypeForResource':{},
        'getReverseRelationshipsOfThisAndParents':{},
        'getPersonalResourceRoots':{},
        'getESMVersion':{},
        'getResourceByName':{},
        'hasXPermission':{},
        'findAllIds':{},
        'addRelationship':{},
        'getReverseRelationshipsOfParents':{},
        'getResourcesReferencePages':{},
        'containsDirectMemberByName':{},
        'getTargetsAsURIByRelationshipForSourceId':{},
        'hasWritePermission':{},
        'deleteResources':{},
        'resolveRelationship':{},
        'getAllPathsToRoot':{},
        'getResourcesWithVisibilityToUsers':{},
        'getReferencePages':{},
        'findAll':{},
        'getExclusivelyDependentResources':{},
        'getAllowedUserTypes':{},
        'getResourcesByNameSafely':{},
        'getResourcesByNames':{},
        'getSourceURIWithThisTargetByRelatiobnship':{},
        'getDataMonitorIfNewer':{},
        'getRelationshipsOfParents':{},
        'getMetaGroupID':{},
        'isValidResourceID':{},
        'getServiceMajorVersion':{},
        'hasReverseRelationship':{},
        'containsDirectMemberByName1':{},
        'getChildNamesAndAliases':{},
        'getResourcesByIds':{},
        'getResourceTypesVisibleToUsers':{},
        'update':{},
        'getRelationshipsOfThisAndParents':{},
        'getPersonalAndSharedResourceRoots':{},
        'getSourcesWithThisTargetByRelationship':{},
        'containsDirectMemberByNameOrAlias1':{},
        'hasReadPermission':{},
        'copyResourceIntoGroup':{},
        'deleteByLocalId':{},
        'getDependentResourceIDsForResourceId':{},
        'getResourceIfModified':{},
        'findById':{},
        'getSourcesWithThisTargetByRelationshipForResourceId':{},
        'getSourcesWithThisTargetByACLRelationship':{},
        'delete':{},
        'getAllPathsToRootAsStrings':{},
        'loadAdditional':{},
        'insertResource':{},
        'insertResources':{},
        'getEnabledResourceIDs':{},
        'getAllUnassignedResourceIDs':{},
        'getSourceURIWithThisTargetByRelationship':{},
        'getResourceById':{},
        'updateACLForResourceById':{},
        'getCorruptedResources':{},
        'unloadAdditional':{},
        'isDisabled':{},
        'getTargetsAsURIByRelationship':{},
        'updateResources':{},
        'deleteByUUID':{},
        'getTargetsByRelationship':{},
        'getSourceURIWithThisTargetByRelationshipForResourceId':{},
        'getTargetsByRelationshipCount':{},
        'getPersonalGroup':{},
        'findByUUID':{},
        'resetState':{},
        'insert':{},
        'getServiceMinorVersion':{},
    },
    'ServerConfigurationService':{
        'getTargetsWithRelationshipTypeForResourceById':{},
        'getTargetsByRelationshipForSourceId':{},
        'getSourcesWithThisTargetByRelationshipCount':{},
        'deleteResource':{},
        'getSourceURIWithThisTargetByRelatiobnshipForResourceId':{},
        'getAllAttachmentOnlyResourceIDs':{},
        'getNamesAndAliases':{},
        'containsDirectMemberByNameOrAlias':{},
        'getTargetsWithRelationshipTypeForResource':{},
        'getReverseRelationshipsOfThisAndParents':{},
        'getPersonalResourceRoots':{},
        'getESMVersion':{},
        'getResourceByName':{},
        'hasXPermission':{},
        'findAllIds':{},
        'addRelationship':{},
        'getReverseRelationshipsOfParents':{},
        'getResourcesReferencePages':{},
        'containsDirectMemberByName':{},
        'getTargetsAsURIByRelationshipForSourceId':{},
        'hasWritePermission':{},
        'deleteResources':{},
        'resolveRelationship':{},
        'getAllPathsToRoot':{},
        'getResourcesWithVisibilityToUsers':{},
        'getReferencePages':{},
        'findAll':{},
        'getExclusivelyDependentResources':{},
        'getAllowedUserTypes':{},
        'getResourcesByNameSafely':{},
        'getResourcesByNames':{},
        'getSourceURIWithThisTargetByRelatiobnship':{},
        'getRelationshipsOfParents':{},
        'getMetaGroupID':{},
        'isValidResourceID':{},
        'getServiceMajorVersion':{},
        'hasReverseRelationship':{},
        'containsDirectMemberByName1':{},
        'getChildNamesAndAliases':{},
        'getResourcesByIds':{},
        'getResourceTypesVisibleToUsers':{},
        'update':{},
        'getRelationshipsOfThisAndParents':{},
        'getPersonalAndSharedResourceRoots':{},
        'getSourcesWithThisTargetByRelationship':{},
        'containsDirectMemberByNameOrAlias1':{},
        'hasReadPermission':{},
        'copyResourceIntoGroup':{},
        'deleteByLocalId':{},
        'getDependentResourceIDsForResourceId':{},
        'getResourceIfModified':{},
        'findById':{},
        'getSourcesWithThisTargetByRelationshipForResourceId':{},
        'getSourcesWithThisTargetByACLRelationship':{},
        'delete':{},
        'getAllPathsToRootAsStrings':{},
        'loadAdditional':{},
        'insertResource':{},
        'insertResources':{},
        'getAllUnassignedResourceIDs':{},
        'getEnabledResourceIDs':{},
        'getSourceURIWithThisTargetByRelationship':{},
        'getResourceById':{},
        'updateACLForResourceById':{},
        'getCorruptedResources':{},
        'unloadAdditional':{},
        'isDisabled':{},
        'getTargetsAsURIByRelationship':{},
        'updateResources':{},
        'deleteByUUID':{},
        'getTargetsByRelationship':{},
        'getSourceURIWithThisTargetByRelationshipForResourceId':{},
        'getTargetsByRelationshipCount':{},
        'getPersonalGroup':{},
        'findByUUID':{},
        'resetState':{},
        'insert':{},
        'getServiceMinorVersion':{},
    },
}

In [5]:
print('hi')

hi


In [ ]:
service_dict.keys()

In [ ]:
service_dict['SecurityEventService']

In [ ]:
REST_URL_BASE = '/www/manager-service/services/'

In [6]:
def generateRestURLs(manager_host_name, service_dict):
    REST_URL_BASE = '/www/manager-service/services/'
    REST_URLs_dict = {}
    for service in service_dict.keys():
        REST_URLs_dict[service] = str('https://{}:8443'.format(manager_host_name) + REST_URL_BASE + \
                                    service + '?wsdl')
    return REST_URLs_dict

In [7]:
rest_URLs = generateRestURLs('esm', service_dict)

In [ ]:
test_url = rest_URLs['GraphService']

In [ ]:
with open('./parse.xml', 'w') as f:
    data = requests.get(test_url, verify=False).content
    f.write(data)

In [ ]:
pwd

In [8]:
def grabWebServiceDescriptions(rest_URLs):
    for service in rest_URLs.keys():
        with open('{}.xml'.format(service), 'w') as f:
            data = requests.get(rest_URLs[service], verify=False).content
            f.write(data)
    return('Wrote WSDL files to current directory.')

In [ ]:
mkdir WSDL

In [ ]:
cd ./WSDL

In [ ]:
grabWebServiceDescriptions(rest_URLs)

In [ ]:
with open('./ActiveListService.xml', 'r') as f:
    data = f.read()

In [ ]:
strip_element = re.compile('\<xs:element name="{}"\>(.*?)\</xs\:element\>'.format('AuthorizationException'), re.DOTALL)
match = strip_element.search(data, re.DOTALL )
raw_elements = match.group()
options_stripper = re.compile('name="(.*?)".*?type="(.*?)"')
options_stripper.findall(raw_elements)

In [9]:
def stripOptions(Call, xml_data):
    strip_element = re.compile('\<xs:element name="{}"\>(.*?)\</xs\:element\>'.format(Call), re.DOTALL)
    match = strip_element.search(xml_data, re.DOTALL )
    raw_elements = match.group()
    if len(raw_elements)== 0:
        strip_element = re.compile('\<complexType name="{}"\>(.*?)\</\complexType\>'.format(Call), re.DOTALL)
        match = strip_element.search(xml_data, re.DOTALL )
        raw_elements = match.group()
    options_stripper = re.compile('name="(.*?)".*?type="(.*?)"')
    options = dict(options_stripper.findall(raw_elements))
    return options

In [10]:
def create_service_dict():
    for Service in service_dict.keys():
        print(Service)
        with open('./{}.xml'.format(Service), 'r') as f:
            print('Opened {}.xml for reading'.format(Service))
            xml_data = f.read()
        for Call in service_dict[Service].keys():
            service_dict[Service][Call] = stripOptions(Call, xml_data)
    return 'Updated service_dict to WSDL specs in current directory.'

In [ ]:
stripOptions('getChildNamesAndAliases', data)

In [ ]:
for Call in service_dict['ActiveListService'].keys():
    service_dict['ActiveListService'][Call] = stripOptions(Call, data)

In [ ]:
service_dict['ActiveListService']

In [12]:
cd ./WSDL/

C:\Users\user\Documents\ArcSight\Dev\WSDL


In [13]:
create_service_dict()

GraphService
Opened GraphService.xml for reading
FileResourceService
Opened FileResourceService.xml for reading
DashboardService
Opened DashboardService.xml for reading
ConAppService
Opened ConAppService.xml for reading
NetworkService
Opened NetworkService.xml for reading
SecurityEventIntrospectorService
Opened SecurityEventIntrospectorService.xml for reading
UserResourceService
Opened UserResourceService.xml for reading
ArchiveReportService
Opened ArchiveReportService.xml for reading
DataMonitorService
Opened DataMonitorService.xml for reading
ResourceService
Opened ResourceService.xml for reading
ViewerConfigurationService
Opened ViewerConfigurationService.xml for reading
QueryViewerService
Opened QueryViewerService.xml for reading
SecurityEventService
Opened SecurityEventService.xml for reading
DrilldownListService
Opened DrilldownListService.xml for reading
ConnectorService
Opened ConnectorService.xml for reading
FieldSetService
Opened FieldSetService.xml for reading
InfoService
Op

'Updated service_dict to WSDL specs in current directory.'

In [ ]:
service_dict

In [ ]:
pwd

In [14]:
cd ../

C:\Users\user\Documents\ArcSight\Dev


In [ ]:
!python ArcSight_ESM_WebServices_API_Client.py -m esm -u admin -s ReportService -c findAll -o {} -j 

In [ ]:
!python ArcSight_ESM_WebServices_API_Client.py -m esm -u admin -s ActiveListService -c findAll -o {} -j 

In [ ]:
!python ArcSight_ESM_WebServices_API_Client.py -m esm -u admin -s CaseService -c findAll -o {} -j 

In [38]:
!python ArcSight_ESM_WebServices_API_Client.py -m esm -u admin -s ActiveListService -c getEntries \
-o (('resourceId':'H-gxUvkwBABCRvFGFq5Z0Ng==')) -j 

usage: example: ArcSight_ESM_WebServices_API_Client.py -m esm -u admin -s ReportService -c findAll -j True
ESM_CLI: error: unrecognized arguments: -o (('resourceId':'H-gxUvkwBABCRvFGFq5Z0Ng=='))


In [49]:
list(('test','test'))

['test', 'test']

In [17]:
doLogin('esm', "admin")

········


C:\Users\user\Anaconda2\lib\site-packages\requests\packages\urllib3\connectionpool.py:789: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


u'K9ZE9pcpVTVCECS_JEAAlzA5tAAbpnUTqYrIeg4QBvY.'

In [22]:
manager_host_name = 'esm' 
user_name = 'admin'
authToken = doLogin(manager_host_name, user_name)
authToken, creationMillis, expirationMillis, userId, sessionToken = getSession(manager_host_name, authToken)

········


C:\Users\user\Anaconda2\lib\site-packages\requests\packages\urllib3\connectionpool.py:789: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
C:\Users\user\Anaconda2\lib\site-packages\requests\packages\urllib3\connectionpool.py:789: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [28]:
response.values()

[u'']

In [146]:
token = doLogin('esm', 'admin')

········


C:\Users\user\Anaconda2\lib\site-packages\requests\packages\urllib3\connectionpool.py:789: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [30]:
token = '9IR4sF1AKtJ0jrHJwg2fYXTdlDleLlJ3Wp5LcUxn1aI.'

In [147]:
authToken, creationMillis, expirationMillis, userId, sessionToken = getSession(manager_host_name, token)

C:\Users\user\Anaconda2\lib\site-packages\requests\packages\urllib3\connectionpool.py:789: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [36]:
options_dict = dict()
Service = 'ReportService'
Call = 'findAllIds'
options_dict['authToken'] = authToken
response = GenericAPICall(manager_host_name, Service, Call, paramater_dict=options_dict, json_return=True)

https://esm:8443/www/manager-service/rest/ReportService/findAllIds?&authToken=9IR4sF1AKtJ0jrHJwg2fYXTdlDleLlJ3Wp5LcUxn1aI.&alt=json


C:\Users\user\Anaconda2\lib\site-packages\requests\packages\urllib3\connectionpool.py:789: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [37]:
response.values()

[{u'rep.return': [u'9025mTEQBABCM3g1s3ZG3-w==',
   u'904C4IScBABCNSxd+xIpUhw==',
   u'90L2FfyUBABCMSK7IUzVNKw==',
   u'90o85TUQBABCc9bYWwaZg4Q==',
   u'91C0UcCcBABCbLLsFWTVnwQ==',
   u'93ObAbfMAABCAiejBIBeTOw==',
   u'93ZMhjgsBABCDWPf4bstZ2A==',
   u'94UkzVxgBABCcPJvsd5GMNQ==',
   u'94jAWS+AAAAAAAAkIrk76ZA==',
   u'94jAWS+BBBBBBBBkIrk76ZA==',
   u'94quJbfMAABCAaejBIBeTOw==',
   u'95p11TeoAABCBFaItwg+LXA==',
   u'96UyttCoBABCE1FfXRe9YMg==',
   u'98cUnxSoBABC2P1fXRe9YMg==',
   u'992WKfyUBABCoiLPQBcQE3A==',
   u'99Dx8jiUBABCvl7PQBcQE3A==',
   u'99ew5P+wAAAAAdWTsqMX6rB==',
   u'9ASeCfyUBABCoNrPQBcQE3A==',
   u'9AXGmURgBABCVypvsd5GMNQ==',
   u'9AhOZURgBABCVtZvsd5GMNQ==',
   u'9Bh4RUhgBABCWTpvsd5GMNQ==',
   u'9ByjRFRgBABCC2TO7e8Yzbw==',
   u'9DS80sA4BABCElF7pd5co-A==',
   u'9DSkKX0YBABCKvEj0BKyo0w==',
   u'9F1GObfMAABCAbOjBIBeTOw==',
   u'9FRKCbfMAABCAZ+jBIBeTOw==',
   u'9GoeVww4BABCCdjh6GMxX-w==',
   u'9GtxkjyUBABC7FrPQBcQE3A==',
   u'9HCADnD8BABD115Cnaz8Y+g==',
   u'9KPlGl-AAABCAAtBjhJARBg

In [35]:
queryActiveLists('esm', authToken)

https://esm:8443/www/manager-service/rest/ActiveListService/findAllIds?&authToken=9IR4sF1AKtJ0jrHJwg2fYXTdlDleLlJ3Wp5LcUxn1aI.&alt=json


C:\Users\user\Anaconda2\lib\site-packages\requests\packages\urllib3\connectionpool.py:789: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


[u'H-gxUvkwBABCRvFGFq5Z0Ng==',
 u'H01U2lkIBABC-8xMqysmSkw==',
 u'H0vSHvlIBABCLqo9LVINKXw==',
 u'H4fRKvVIBABCBi+LAelEAIQ==',
 u'H5Z18FEgBABDxUaFzVZ5nLg==',
 u'H5pHS1kgBABCJ7qyV5JI2rA==',
 u'H5wtztCoBABCC01fXRe9YMg==',
 u'H77VcOgQBABCAKcikqcW9YA==',
 u'H7GfZOfMAABCAEkfFgoQ8IA==',
 u'H7nDOZB0BABCB337GCqa7qw==',
 u'H8sfU1kgBABCJ7zdsTR+9Ow==',
 u'H9GpTvkwBABCRu7zLc1-G9w==',
 u'HBfLjoEMBABCpmWsJ2iDoUA==',
 u'HC3RPPe0AABCAATRMay889A==',
 u'HCgkx8kgBABC64ZgVl-F9oA==',
 u'HD2vbOfMAABCAE0fFgoQ8IA==',
 u'HDr-e4kwBABCNq2TjR+aeGg==',
 u'HFz6JXBoBABCKyxNV2v9A2A==',
 u'HHMDelDABABCX36qVVkqC2w==',
 u'HIYBKPe0AABCAAdGjSMnsPw==',
 u'HJHHP1kgBABCJ7UB8oDVnSw==',
 u'HJa0VwCQBABCBZZiy2jXu3A==',
 u'HJodRvkwBABCRt3SmrXvJ7A==',
 u'HKQrbgj8BABCGnOcQ2p-hxw==',
 u'HKRIgjT8BABCAgSy8B+oWwA==',
 u'HLZRc9yYBABC-lHtlf3-f0Q==',
 u'HLgSPXBoBABCK9RNV2v9A2A==',
 u'HLoaacUUBABCwfCaRWQMIeA==',
 u'HNsSQZB0BABCBD37GCqa7qw==',
 u'HOZkmoTgBABDFn3c1OouxLw==',
 u'HPPhtfi4BABCZpzyf0B-0Gg==',
 u'HQsF2KR8BABCB-dHnQejEAg==',
 u'HRC9V

In [127]:
GenericAPICall('esm', 'ActiveListService', 'GetEntries', 'resourceId=H-gxUvkwBABCRvFGFq5Z0Ng==')

ValueError: too many values to unpack

In [149]:
def GenericAPICall(manager_host_name, Service, Call, authToken, parameter_kv_pairs=None, json_return=True):
    if parameter_kv_pairs==None:
        resource_request_json = \
        'https://{}:8443/www/manager-service/rest/{}/{}?&authToken={}&alt=json'\
        .format(manager_host_name, Service, Call, authToken) 
    else:
        paramaters=''
        parameter_kv_pairs = parameter_kv_pairs.split(',')
        print(parameter_kv_pairs)
        kv_pairs = {}
        for pair in parameter_kv_pairs:
            k = pair.split('=', 1)
            kv_pairs[k[0]] = k[1]
        kv_pairs['authToken'] = authToken
        for entry in kv_pairs.keys():
            paramaters +='&' + entry + '=' + kv_pairs[entry]
        print(kv_pairs)
        resource_request_json = \
        'https://{}:8443/www/manager-service/rest/{}/{}?{}&alt=json'\
        .format(manager_host_name, Service, Call, paramaters)
    print(resource_request_json)
    retrieve_list = requests.get(resource_request_json, verify=False)
    if not json_return:
        return retrieve_list
    else:
        response = json.loads(retrieve_list.content)
        return response

In [150]:
GenericAPICall('esm', 'ActiveListService', 'getEntries', authToken, 'resourceId=H-gxUvkwBABCRvFGFq5Z0Ng==')

['resourceId=H-gxUvkwBABCRvFGFq5Z0Ng==']
{'authToken': 'GFdKcXhS01yCY4zP4r4v48nxGUuk_JLFmyH9zddiREQ.', 'resourceId': 'H-gxUvkwBABCRvFGFq5Z0Ng=='}
https://esm:8443/www/manager-service/rest/ActiveListService/getEntries?&authToken=GFdKcXhS01yCY4zP4r4v48nxGUuk_JLFmyH9zddiREQ.&resourceId=H-gxUvkwBABCRvFGFq5Z0Ng==&alt=json


C:\Users\user\Anaconda2\lib\site-packages\requests\packages\urllib3\connectionpool.py:789: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


{u'act.getEntriesResponse': {u'act.return': {u'columns': [u'name',
    u'targetAddress',
    u'targetZone']}}}